**REVISIÓN DE CONDICIONES PARA PARTICIPAR EN INSTAGRAM SOY UN POLLITO**

**Instrucciones**
- Seguirme en Instagram
- Escribir en comentario: Soy un pollito 🐥
- Si etiquetas hasta 5 personas tienes más posibilidades de ganar
- Si las personas etiquetadas me siguen tienes aún más posibilidades de ganar

**Librerias**

In [37]:
import re
import numpy as np
import pandas as pd
import gspread as gs
import gspread_dataframe as gd

import google.colab.auth as auth
from google.colab import auth

In [38]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [39]:
# Cliente
client = gs.authorize(creds)

In [40]:
# Leer mi spreadsheet
spreadsheet = client.open_by_key('1LIoRzpLFIcSvZPnyy7qSZDYhVOrIzhqGndSD81u9PJE')

In [41]:
# Leer sheets (hojas)
sheet1 = spreadsheet.worksheet('followers')
sheet2 = spreadsheet.worksheet('comments')

In [42]:
# Convertir en dataframes
df_sheet1 = gd.get_as_dataframe(sheet1)
df_sheet2 = gd.get_as_dataframe(sheet2)

**Analizar datos**

**1. Seguidores**

In [43]:
followers = (
    df_sheet1
    .loc[:, ['followers']]
    .query('followers.notnull()')
    .query('~followers.str.contains(r"AM$|PM$")')
    .rename(columns={'followers': 'usuario'})
)

In [44]:
followers.shape

(3964, 1)

In [45]:
followers

,usuario
0,katherinebolanosh
2,brando_torresy
4,carlos138531
6,yulaiju
8,jhosyoshinn
...,...
7918,diegobmon
7920,lordgustavol
7922,luisalbertobenaventediaz
7924,luisenriquecorp


**2. Comentarios**

In [46]:
df_sheet2.shape

(1926, 1)

In [47]:
# Convertir comentarios en formato de tabla

comments = (
    df_sheet2
    .loc[:, ['comentario']]
    .query('comentario.notnull()')
    .assign(comentario = lambda x: x['comentario'].str.lower())
    .query('~comentario.str.contains(r"translation\\b|replies\\b|reply\\b", na=False)',
           engine='python')
    .assign(
        lead = lambda x: x['comentario'].shift(-1)
    )
        .loc[:, ['comentario', 'lead']]
    .rename(columns={
        'comentario': 'usuario', 'lead': 'comentario'
    })
    .query("usuario.str.match(r'^[\\w.]+$', na=False)", engine='python')
)

In [48]:
comments

,usuario,comentario
0,yowel_of,soy un pollito 🐥 @xenia_cordovaa @josef.cc17 @...
5,xenia_cordovaa,@yowel_of yo también son un pollito 🐤
9,analucerocp,amooooo como finalizaste el video tkm amo tu a...
14,analucerocp,@analucerocp nunca dejes de hacerlo 😍
18,layeraxd,@analucerocp nunca dejare de hacerlo!! wiii! s...
...,...,...
1907,dellymiluzka,soy un pollito 🐣 pio pio. necesito curso de r ...
1911,davodex_,soy un pollito @dg_rhl + @claudiadesme + @sad....
1915,alexandra__lucero,soy un 🐥 @kathycr__ @lissana_strauss95 @77777k...
1918,ajhonnny.y,soy un pollito 🐤 🐥🐣@katerinmucha @aquinohuayll...


In [49]:
comments['usuario'].drop_duplicates().count()

np.int64(347)

**3. Filtro emoji pollito**

In [50]:
# Comentario debe contener algún emoji de pollito 🐤,🐣,🐥

primer_filtro = (
    comments
    .query('comentario.str.contains("🐤|🐣|🐥")')
    .sort_values(['usuario'])
    .drop_duplicates()
)

In [51]:
comentarios_eliminados = (
    comments
    .merge(primer_filtro.drop(['comentario'], axis=1).assign(filtro='Aprobado'),
           on=['usuario'], how='left')
    .query('filtro.isnull()')
)

In [52]:
primer_filtro

,usuario,comentario
714,93andre_8,soy un pollito 🐣@si.segura @migatitoeslindo
1239,_aleabadd,soy un pollito🐣haha
1208,_liv164,soy un pollito 🐥 🐣 @cecilia._valenzuela @ana.ep
1421,a.estefany.27,soy un pollito 🐥 @its.ani_fer @toska_0006 @ani...
1453,abisnunu,🐣🐣 @klvswag @maidoydoida @karlanehuza27 @aixam...
...,...,...
1838,yosoygeraldine,soy un pollito 🐣🐤🐥 @brigette_tm @elle_marjorie...
617,yovanyortecho,soy un pollito 🐥
0,yowel_of,soy un pollito 🐥 @xenia_cordovaa @josef.cc17 @...
1061,yvonsn,soy un pollito 🐣


In [53]:
primer_filtro['usuario'].drop_duplicates().count()

np.int64(310)

In [54]:
comentarios_eliminados

,usuario,comentario,filtro
8,_johnaguilar_,@juan_aurelio03 ya se sabía ya,NaN
12,hope_hope1997,@xtrilcex suerte bebeeeeee,NaN
17,nestorjesus0601,@liz_valb ✍️,NaN
21,hatalian,@xio.yam.03 también quiero ser un pollito,NaN
23,aurorisisima,@harold.caballero_ 🫶🏼,NaN
30,_giancarlol,@anintrin eres una polla,NaN
36,samantha_qs,@susivaler pollito ganador,NaN
41,maira_wh.23,@anmi9_14 que ganeeees❤️❤️,NaN
60,frankcabanillas03,@jepherson_pr jajajajajajajajajjaja,NaN
80,jhonny_guille,@rmaxxyy soy un pollito🤭,NaN


In [55]:
comentarios_eliminados['usuario'].drop_duplicates().count()

np.int64(37)

**4. Filtro de seguidores de Miss Yera**

In [56]:
# Comentario debe contener la frase "Soy un pollito"

segundo_filtro = (
    primer_filtro
    .merge(followers.assign(seguidor='Si'), how='left', on=['usuario'])
    .query('seguidor.notnull()')
    .drop(['seguidor'], axis=1)
    .assign(comentario = lambda x: x['comentario'].str.strip())
    .assign(comentario = lambda x: x['comentario'].str.replace(r'\s+', ' ', regex=True))
    .query('comentario.str.contains("soy un pollito")')
    .assign(arrobas = lambda x: x['comentario'].str.count('@')+1)
)

In [57]:
comentarios_eliminados = (
    primer_filtro
    .merge(segundo_filtro.drop(['comentario'], axis=1).assign(filtro='Aprobado'),
           on=['usuario'], how='left')
    .query('filtro.isnull()')
)

In [58]:
segundo_filtro

,usuario,comentario,arrobas
1,_aleabadd,soy un pollito🐣haha,1
7,adrianafajardor,soy un pollito 🐥,1
16,ahirtonparedes31,soy un pollito 🐥 @christian_zch,2
18,ale.paredesgo,soy un pollito 🐣 @milagroskyara @cesar_cc95 @c...,4
22,alex.velarde16,soy un pollito 🐥 @jezhuquiroz @lindero2317 @da...,7
...,...,...,...
356,xtrilcex,soy un pollito 🐣 😎 @fiorellaarcoss @jeminaguer...,6
364,yosi.sanz,soy un pollito 🐣,1
366,yovanyortecho,soy un pollito 🐥,1
367,yowel_of,soy un pollito 🐥 @xenia_cordovaa @josef.cc17 @...,6


In [59]:
segundo_filtro['usuario'].drop_duplicates().count()

np.int64(95)

In [60]:
comentarios_eliminados

,usuario,comentario,arrobas,filtro
0,93andre_8,soy un pollito 🐣@si.segura @migatitoeslindo,NaN,NaN
2,_liv164,soy un pollito 🐥 🐣 @cecilia._valenzuela @ana.ep,NaN,NaN
3,a.estefany.27,soy un pollito 🐥 @its.ani_fer @toska_0006 @ani...,NaN,NaN
4,abisnunu,🐣🐣 @klvswag @maidoydoida @karlanehuza27 @aixam...,NaN,NaN
5,acardenas.9.10,soy un pollito 🐥@padmaes11 @trikimazana @malma...,NaN,NaN
...,...,...,...,...
415,yisuchan,soy un pollito 🐣🐣 @florsteneri @urd.verdandi.s...,NaN,NaN
416,yiulianada,soy un pollito 🐥 @danysahernandezcoral @jade08...,NaN,NaN
417,yonathan.9.8,soy un pollito 🐥,NaN,NaN
419,yosoygeraldine,soy un pollito 🐣🐤🐥 @brigette_tm @elle_marjorie...,NaN,NaN


In [61]:
comentarios_eliminados['usuario'].drop_duplicates().count()

np.int64(215)

**5. Bono de personas etiquetadas que siguen a Miss Yera**

In [62]:
# Función para extraer personas etiquetadas por cada comentarios

def extraer_etiquetas(df):

    def encontrar_etiquetas(text):
        etiquetas = re.findall(r'@([\w.]+)', text)
        return etiquetas

    df['etiquetas'] = df['comentario'].apply(encontrar_etiquetas)
    return df

In [63]:
# Validar si personas etiquetas siguen a Miss Yera

bono_etiquetas = (
                extraer_etiquetas(segundo_filtro)
                .explode('etiquetas')
                .loc[:, ['usuario', 'etiquetas']]
                .drop_duplicates()
                .merge(followers.rename(columns={'usuario': 'seguidor'}),
                       how='left', left_on='etiquetas', right_on='seguidor')
                .assign(
                    #seguidor = lambda x: x['seguidor'].astype(str),
                    arrobas_seguidores = lambda x: np.where(x['seguidor'].notnull(), 1, 0)
                )
                #.assign(seguidor = lambda x: np.where(x['seguidor']=='nan','',x['seguidor']))
                .groupby(['usuario'], dropna=False)
                .agg(
                     #seguidores=('seguidor', ' | '.join),
                     arrobas_seguidores = ('arrobas_seguidores', 'sum')
                )
                .reset_index()
)

In [64]:
bono_etiquetas

,usuario,arrobas_seguidores
0,_aleabadd,0
1,adrianafajardor,0
2,ahirtonparedes31,0
3,ale.paredesgo,0
4,alex.velarde16,0
...,...,...
90,xtrilcex,0
91,yosi.sanz,0
92,yovanyortecho,0
93,yowel_of,0


**6. Total de chances por participante**

In [65]:
# Calcular cantidad de chances por participantes (máximo 5)

chances = (
    segundo_filtro
    .merge(bono_etiquetas, how='left', on=['usuario'])
    .assign(total_arrobas = lambda x: x['arrobas'] + x['arrobas_seguidores'])
    .drop(['etiquetas','arrobas','arrobas_seguidores'], axis=1)
    .drop_duplicates()
    .drop(['comentario'], axis=1)
    .groupby(['usuario'])
    .agg(total_arrobas = ('total_arrobas', 'max'))
    .reset_index()
    .sort_values(['total_arrobas'], ascending=False)
    .assign(total_arrobas = lambda x: np.where(x['total_arrobas']>5, 5, x['total_arrobas']))
    .reset_index(drop=True)
)

In [66]:
chances

,usuario,total_arrobas
0,raquelburgos12,5
1,solanovidurrizaga,5
2,sanderson_albeniz,5
3,lu.chirinos17,5
4,alex.velarde16,5
...,...,...
90,redes_ccovis,1
91,sevencrizoficial,1
92,yosi.sanz,1
93,yovanyortecho,1


**7. Repetir nombre de usuario por cantidad de chances**

In [67]:
# Repetir nombre de usuario por cantidad de chances

lista_final = []
for index, row in chances.iterrows():
  for _ in range(int(row['total_arrobas'])):
    lista_final.append(row)
lista_final = pd.DataFrame(lista_final)

In [68]:
lista_final

,usuario,total_arrobas
0,raquelburgos12,5
0,raquelburgos12,5
0,raquelburgos12,5
0,raquelburgos12,5
0,raquelburgos12,5
...,...,...
90,redes_ccovis,1
91,sevencrizoficial,1
92,yosi.sanz,1
93,yovanyortecho,1


**8. Final: Seleccionar top 10 índices de manera random**

In [69]:
# Lista de ganadores

lista_final.sample(n=10)['usuario']

,usuario
26,fdianar.24
80,jhoavega0911
54,josueabrahamcr
88,fivavix.js
7,cris_cossio
20,lasrutasperuanas
89,gilmarmamani
30,luteciox
50,izispdc
4,alex.velarde16
